# Symposium on Molecular Interactions Tutorials
## What is this?? (Session 2).
This is a Jupyter notebook. It runs Python, but also allows it to be interspersed with Markdown-formatted text, for a nice mixture of documentation and code.

The particular material below is Session 2 of the tutorials on free energy calculations and OpenMM, from Day 1 of the Symposium on Molecular Interactions (2017).

This content is available on the [symposium github repository](https://github.com/QCMM/workshop2017) at [https://github.com/QCMM/workshop2017](https://github.com/QCMM/workshop2017).

This material is a set of slides intended for presentation with RISE (for presentations from Jupyter notebooks). While it may be useful without RISE, it will also likely appear somewhat less verbose than it would if it were intended for use in written form.

## Authors/sources/acknowledgments
Author: David L. Mobley (UCI)

### Sources
- Much of content adapted from materials I have developed or am developing for my [Drug Discovery Computing Techniques class](https://github.com/mobleylab/drug-computing).

### Acknowledgments
- John Chodera and Nathan Lim for help with OpenMM
- OpenEye for help with OpenEye software, especially James Haigh

# Outline of this afternoon's second session
1. Some brief OpenMM basics (what's going on under the hood in Yank)
2. A simple example of a density calculation which stops when converged
3. Analyzing density results in this notebook
4. SMIRNOFF force fields, modifying the density calculation to use them

**To follow along**/run code: Open this Jupyter notebook in `solvation_free_energies/Session2.ipynb` via `jupyter notebook Session2.ipynb`. (See [prerequisites](../prerequisites.md) if you aren't using preinstalled software.) This is cumulative, building on `Session1.ipynb`.



# OpenMM is more of a simulation toolkit than a simulation package
- Easy-to-use Python API
- Very fast calculations on GPUs (but slow on CPUs)
- Really easy to implement new techniques, do new science
- Key ingredients in a calculation:
    - `Topology`
    - `System`
    - `Simulation` (takes `System`, `Topology`, `Integrator`; contains positions)

## `Topology`: Chemical composition of your system

- Atoms, bonds, etc.
- Can be loaded from some common file formats such as PDB, mol2
- Can be created from OpenEye molecule via [`oeommtools`](https://github.com/oess/oeommtools), such as [`oeommtools.utils.oemol_to_openmmTop`](https://github.com/oess/oeommtools/blob/master/oeommtools/utils.py#L17)
    - Side note: An OE "molecule" can contain more than one molecule, so can contain protein+ligand+solvent for example
- Tangent: Try to retain bond order info if you have it (e.g. from a mol2)

In [4]:
# Example Topology generation from a couple mechanisms here

# Load a PDB
from simtk.openmm.app import PDBFile
pdb = PDBFile('sample_files/T4-protein.pdb')
t4_topology = pdb.topology

In [8]:
# Load a mol2: MDTraj supports a variety of file formats including mol2
import mdtraj
traj = mdtraj.load('sample_files/mobley_20524.mol2')
# MDTraj objects contain a Topology, but an MDTraj topology; they support conversion to OpenMM
traj.topology.to_openmm()

/Users/dmobley/anaconda3/envs/workshop_test/lib/python3.6/site-packages/pandas/io/parsers.py:2218: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/Users/dmobley/anaconda3/envs/workshop_test/lib/python3.6/site-packages/pandas/io/parsers.py:2220: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


In [ ]:
# Load AMBER

# Load GROMACS

# Load an OEMol and convert (note advantage over MDTraj for bond order, etc.)

## Probably should have some visualization, note that they can check these out using VMD?

## `System`: Your parameterized system
- Often generated by `createSystem`, but requires OpenMM know how to assign parameters
  - Easy for standard biomolecules (proteins, nucleic acids), waters ions
  - OpenMM FFXML files used; available for many common FFs
  - More complex for general small molecules
- Can also be loaded from common file formats such as GROMACS, AMBER
  - useful if you set up for AMBER or GROMACS
- We have new open forcefield effort that provides new force fields with an `openforcefield.createSystem` operator; generates OpenMM Systems.

In [2]:
## From OpenMM Docs: http://docs.openmm.org/latest/userguide/application.html#running-simulations
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

pdb = PDBFile('input.pdb')
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(10000)

# Example System creation here

# Do something (water?) using OpenMM FFXML

# Load something canned from AMBER or GROMACS

# Prep a molecule with GAFF using openmoltools?

## Probably should have some visualization, note that they can check these out using VMD?

## `Simulation`: The system, topology, and positions you're simulating, under what conditions
- Could be for energy minimization, or different types of dynamics
- Has an integrator attached (even if just minimizing), including temperature
- `context` -- including positions, periodic box if applicable, etc.
- If dynamics, has:
  - timestep
  - velocities
- potentially also has reporters which store properties like energies, trajectory snapshots, etc.

In [3]:
# Show an example energy minimization of one of the systems considered above

# A simple example of a density calculation which stops when converged


In [1]:
from solvationtoolkit.solvated_mixtures import *
mixture = MixtureSystem('density/mixture')
mixture.addComponent(label='phenol', smiles='c1ccc(cc1)O', number=1)
mixture.addComponent(label='toluene', smiles='Cc1ccccc1', number=10)
mixture.addComponent(label='cyclohexane', smiles='C1CCCCC1', number=500)
#Generate output files for AMBER
mixture.build(amber = True)

/Users/dmobley/anaconda3/envs/workshop_test/lib/python3.6/site-packages/pandas/io/parsers.py:2218: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/Users/dmobley/anaconda3/envs/workshop_test/lib/python3.6/site-packages/pandas/io/parsers.py:2220: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())



# Mixture 

tolerance 2.000000
filetype pdb
output /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmplloo4y8r/tmp27fos4rb.pdb
add_amber_ter


structure /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmplloo4y8r/tmpo7tbdzr9.pdb
  number 1 
  inside box 0. 0. 0. 43.668714 43.668714 43.668714
end structure

structure /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmplloo4y8r/tmpik9u18as.pdb
  number 10 
  inside box 0. 0. 0. 43.668714 43.668714 43.668714
end structure

structure /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmplloo4y8r/tmp7pbd1ul8.pdb
  number 500 
  inside box 0. 0. 0. 43.668714 43.668714 43.668714
end structure


source leaprc.gaff
source oldff/leaprc.ff99SB
ZWD = loadmol2 in0.mol2
ZEF = loadmol2 in1.mol2
ZZW = loadmol2 in2.mol2
box = loadPdb tbox.pdb
loadamberparams in0.frcmod
loadamberparams in1.frcmod
loadamberparams in2.frcmod
setbox box centers
saveAmberParm box out.prmtop out.inpcrd
quit



## There's a script version of this which has equilibration, etc... deposited ...

# Analyzing density results in this notebook

# SMIRNOFF force fields, modifying the density calculation to use them

In [5]:
import pandas